In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"
import keras

In [ ]:
from pathlib import Path
files = [file for file in Path("dataset_by_aspect_ratio/general").rglob("**/*.jpg")]
train_folder = Path("dataset_by_aspect_ratio/train")
valid_folder = Path("dataset_by_aspect_ratio/valid")

In [ ]:
len(files)

In [ ]:
import tensorflow as tf
import numpy as np
imgs = []
for file in files:
    img = tf.io.read_file(str(file.absolute()))
    img = tf.image.decode_image(img, channels=1)
    img = tf.image.resize(img, (512, 512))
    imgs.append(img)
imgs = np.array(imgs)


In [ ]:
imgs.shape

In [ ]:
model = keras.models.load_model("models/circle-versus-lack_of_fusion_best_val_acc.keras", compile=False)

In [ ]:
result = model.predict(imgs)

In [ ]:
result_arry = [(float(a), float(b)) for a, b in result]

In [ ]:
len(result_arry)

In [ ]:
len(set(result_arry))

In [ ]:
duplicate_indices = []
result_len = len(result_arry)
for index_a in range(result_len):
    for index_b in range(index_a+1, result_len):
        if result_arry[index_a] == result_arry[index_b]:
            duplicate_indices.append((index_a, index_b))

In [ ]:
duplicate_indices

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
indices = set()
for index_a, index_b in duplicate_indices:
    indices.add(index_a)
    indices.add(index_b)
    img_a, img_b = Image.open(files[index_a]), Image.open(files[index_b]) 
    plt.imshow(img_a)
    plt.show()
    plt.imshow(img_b)
    plt.show()
    print(files[index_a].absolute())
    print(files[index_b].absolute())

In [ ]:
indices

In [ ]:
for index in indices:
    file = files[index]
    file_name = file.name
    label = file.parent.name
    file_to_remove = train_folder / label /file_name
    if not file_to_remove.exists():
        file_to_remove = valid_folder / label /file_name
    file_to_remove.unlink()